<a href="https://colab.research.google.com/github/Pranpriya04/Colab-DS-Selected-Blockchain/blob/main/Bitcoin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import hashlib
import binascii
import datetime
import collections
import Crypto.Random
from Crypto.Hash import SHA
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5

# Client

In [14]:
class Client:
    def __init__(self):
        random_gen = Crypto.Random.new().read
        self.pri_key = RSA.generate(1024, random_gen)
        self.pub_key = self.pri_key.publickey()
        self.signer = PKCS1_v1_5.new(self.pri_key)

    def identity(self):
        return binascii.hexlify(
            self.pub_key.exportKey(format='DER')
        ).decode('ascii')

# Transaction

In [15]:
class Transaction:
    def __init__(self, sender, receiver, value):
        self.sender = sender
        self.receiver = receiver
        self.value = value
        self.time = datetime.datetime.now()

    def to_dict(self):
        if self.sender == "Genesis":
            identity = "Genesis"
        else:
            identity = self.sender.identity()
        return collections.OrderedDict({
            'sender': identity,
            'receiver': self.receiver.identity() if self.receiver != "Genesis" else "Genesis",
            'value': self.value,
            'time': self.time.strftime("%Y-%m-%d %H:%M:%S")
        })

    def sign_transaction(self):
        if self.sender == "Genesis":
            return None
        signer = PKCS1_v1_5.new(self.sender.pri_key)
        h = SHA.new(str(self.to_dict()).encode('utf8'))
        return binascii.hexlify(signer.sign(h)).decode('ascii')


def display_transaction(transaction):
    d = transaction.to_dict()
    print("sender:", d['sender'])
    print("receiver:", d['receiver'])
    print("value:", d['value'])
    print("time:", d['time'])
    print("-----")

# Block

In [16]:
class Block:
    def __init__(self):
        self.verified_transactions = []
        self.previous_block_hash = ""
        self.Nonce = 0
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        sha = hashlib.sha256()
        sha.update(str(self.previous_block_hash).encode('utf-8') +
                   str(self.Nonce).encode('utf-8'))
        return sha.hexdigest()

    def mine(self, difficulty):
        while self.hash[0:difficulty] != "0" * difficulty:
            self.Nonce += 1
            self.hash = self.calculate_hash()
        return self.Nonce


def dump_blockchain(block_list):
    print("Number of blocks in the chain:", str(len(block_list)))
    for i, block in enumerate(block_list):
        print("Block #", i)
        for tran in block.verified_transactions:
            display_transaction(tran)

# Main

In [17]:
if __name__ == '__main__':
    A = Client()
    B = Client()
    C = Client()

    print("Client A identity:", A.identity())
    print("Client B identity:", B.identity())
    print("Client C identity:", C.identity())
    print("="*50)


    block_list = []
    transaction_list = []
    transaction_index = 0

    genesis = Block()
    genesis.previous_block_hash = None
    genesis.Nonce = None
    t0 = Transaction("Genesis", A, 50)
    genesis.verified_transactions.append(t0)
    digest = hash(str(genesis))
    last_hblock = digest
    block_list.append(genesis)

    t1 = Transaction(A, B, 20)
    t2 = Transaction(B, C, 10)
    t3 = Transaction(C, A, 5)

    transaction_list.extend([t1, t2, t3])

    new_block = Block()
    for t in transaction_list:
        new_block.verified_transactions.append(t)
        transaction_index += 1

    new_block.previous_block_hash = last_hblock
    new_block.mine(difficulty=2)
    digest = hash(str(new_block))
    last_hblock = digest
    block_list.append(new_block)

    dump_blockchain(block_list)

Client A identity: 30819f300d06092a864886f70d010101050003818d0030818902818100a711d66ef2e9d99eef6f52c3ff4319346d61fda4e67854267dcba29c7c63f131c4afeee6f61d90948ac5074d727f5d55766189af542136eaa96b63bdee4b5edb3a59c98404b5c27492c85588e26e0ff5bfbe8f012181dd4b5420b220407e50584cf6371701e638048a633ac9ce4eff891f05edeeda66a8d7695aa76b2b8afde50203010001
Client B identity: 30819f300d06092a864886f70d010101050003818d0030818902818100b598d7d3906dc2a62a0c5a00e6c0dbc3b2ff595ff2f10698ea0b16a65fee2950abca8ae12419844d9f3137a1b3b2c9877258fc1cc73a7c47cc89b51a6bf75cee37629a1538ad30472a0197c5e82a9a5b144214d4f62a89455320054ffafd53223adaefecad32da6380e80c09b4d735c2207e3a9ef409641600fd44e1fd9baed90203010001
Client C identity: 30819f300d06092a864886f70d010101050003818d0030818902818100b0b607a88925e2a83cec0db1542dc09ece62c35869541d8e5a2491983230a470e3d51248e8c88e8cd6d08444a28152413de01f1caaaf305f03f6a55323f1e00c4d76109cc3c955d6c60f101629653bdb601b9435f670fa7c3ad91d66b69e572c6e16ab19ca6010f09a960e1b2726eebcb255c61a4ce